In [7]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import threading
import os
import socket
from multiprocessing import Process, Value, Array, Lock

base = BaseOverlay("base.bit")

In [8]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

//Add a C++ function to reset 0,1,4,5 LOW and 2,3,6,7 HIGH the GPIO pins on the chosen PMOD
void lh_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        if (j==0 || j==1 || j==4 || j==5) {
            write_gpio(j, 0);
        }
        else {
            write_gpio(j,1);
        }
    }
}

In [9]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpioB(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpioB(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpioB(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpioB(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [10]:
#LED_Status = [0, 0, 2, 3] #Default Status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness]
change_LED = False
#start_LED = False
press = False
#Send_Status = False
compare_to = [0, 0, 1, 1, 0, 0, 1, 1] #this is if no buttons pressed!
iteration = -1 #To see where in the user_input array the specific input is remembered
user_input = [] #user_input[iteration] to see most recent addition.
running = True
threads = []

sleep_time = 1
frequency = 15 #for the PWM Brightness
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
PWM_run_time = frequency*sleep_time

reset_gpioB()

def LED(LED_Status):
    global PWM_on, PWM_off, PWM_run_time, sleep_time
    if LED_Status[0] == 0: #IF OFF
        time.sleep(sleep_time)
    else: #IF ON
        if LED_Status[3] == 3: #full bright (NO PWM)
            if LED_Status[2] == 0: #If NO PWM & Cycle Color
                for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                    write_gpioB(i,1) #turn on  LED
                    time.sleep(sleep_time) #sleep with ON
                    write_gpioB(i,0) #turn off LED
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                        time.sleep(sleep_time)
            else: #IF NO PWM & Solid Color
                write_gpioB(LED_Status[2],1)
                time.sleep(sleep_time)
                if LED_Status[1] == 1: #if supposed to blink, sleep
                        write_gpioB(LED_Status[2],0)
                        time.sleep(sleep_time)
        else: #If NOT full brightness (PWM needed)
            if LED_Status[2] == 0: #PWM & cycle thru colors
                for i in range(1,4):
                    for j in range(PWM_run_time):
                        write_gpioB(i,1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpioB(i,0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If blinking
                        time.sleep(sleep_time)
            else:
                for i in range(PWM_run_time):
                    write_gpioB(LED_Status[2],1) #turn on LED
                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                    write_gpioB(LED_Status[2],0) #turn off LED
                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                if LED_Status[1] == 1: #If Blinking
                    time.sleep(sleep_time)

def keypad(key_list = []):
    global user_input, iteration
    lh_gpio()
    #find which key was pressed
    #print(key_list)
    if key_list[0] == 1:
        user_input.append('147*')
    elif key_list[1] == 1:
        user_input.append('2580')
    elif key_list[4] == 1:
        if key_list[2] == 0:
            user_input.append('3')
        elif key_list[3] == 0:
            user_input.append('6')
        else:
            user_input.append('9#')
    else: #key_list[5] == 1
        if key_list[2] == 0:
            user_input.append('A')
        elif key_list[3] == 0:
            user_input.append('B')
        else:
            user_input.append('CD')
    print('New Input Detected: ',user_input[iteration])
    
def button_press():
    global compare_to, iteration, press
    #see if a button was pressed
    value = []
    for i in range(8):
        value.append(read_gpio(i))
    if compare_to != value: #if a keypad input was detected
        iteration = iteration+1
        keypad(value) #find out which input was detected
        press = True
    #else: 
    
def Change_LED_Status(LED_Status_change, _Lc1, Send_change, _Lc2):#SEND STATUS!
    global compare_to, user_input, iteration, change_LED
    LED_Status_New = []
    with _L1:
        for i in range (4):
            LED_Status_New.append(LED_Status_change[i])
    if user_input[iteration] == '147*': #ON/OFF
        if LED_Status_New[0] == 0:
            LED_Status_New[0] = 1
        else:
            LED_Status_New[0] = 0
    elif user_input[iteration] == '2580': #Solid/Blink
        if LED_Status_New[1] == 0:
            LED_Status_New[1] = 1
        else:
            LED_Status_New[1] = 0
    elif user_input[iteration] == 'A': #RED
        LED_Status_New[2] = 2
    elif user_input[iteration] == 'B': #GREEN
        LED_Status_New[2] = 3
    elif user_input[iteration] == '3': #BLUE
        LED_Status_New[2] = 1
    elif user_input[iteration] == '6': #CYCLE
        LED_Status_New[2] = 0
    elif user_input[iteration] == 'CD': #BRIGHTER
        if LED_Status_New[3] < 3:
            LED_Status_New[3] = LED_Status_New[3]+1
    else: # (user_input[iteration]=='9#') #DIMMER
        if LED_Status_New[3] > 0:
            LED_Status_New[3] = LED_Status_New[3]-1
    #Now check to see if the button input changed LED state
    #(e.g. was brightness already max so does it need to send message to other pynq?)
    with _Lc1:
        if LED_Status_change != LED_Status_New:
            print('New status update to be sent')
            reset_gpioB()
            for i in range(4):
                LED_Status_change[i] = LED_Status_New[i]
            with _Lc2:
                Send_change.value = 1
            change_LED = True
        else:
            print('User input does not result in any changes. Nothing will be sent.')

def LOOP(_l, LED_Status_l,send_l,running_l,_Ll1,_Ll2,_Ll3): #ADD RUNNING WITH LOCK????
    global press, sleep_time, change_LED
    with _Ll3:
        continue_LED = running_l.value
    while continue_LED == 1: #running.value = 1:
        resource_available = _l.acquire(False)
        if resource_available == True:
            with _Ll1:
                LED_blinking = LED_Status_l
            change_LED = False
            while change_LED == False and continue_LED == 1:
                LED(LED_blinking)
                with _Ll3:
                    if running_l.value == 0:
                        continue_LED = 0
            print('LED() temporarily terminated.')
            _l.release()
            #time.sleep(sleep_time) #only sleep if NOT PWM
        else:
            #print('Worker {} is searching for keypad input.'.format(num)) #USE TO FIND DEADLOCKS & adjust sleep time!
            button_press()
            if press == True:
                time.sleep(sleep_time) #add a delay so it doesn't register as multiple inputs!
                #change value of
                Change_LED_Status(LED_Status_l, _Ll1, send_l, _Ll2)
                press = False
            else:
                with _Ll3:
                    #print('checking running value: ',running_l.value)
                    if running_l.value == 0:
                        continue_LED = 0
                time.sleep(sleep_time) #update time?
            lh_gpio()
            #time.sleep(sleep_time*.5)


def Start_Keypad(start_k,LED_Status_k,send_k, running_k,_Lk0,_Lk1,_Lk2,_Lk3):    
    global threads, start_LED
    while True:
        with _Lk0:
            if start_k.value == 1:
                break
        time.sleep(sleep_time)
    lh_gpio() #reset gpio to same as "compare_to"
    reset_gpioB()
    fork = threading.Lock()
    for i in range(2):
        #fork is the shared lock
        #i is the user (led number too)
        t = threading.Thread(target=LOOP, args=(fork, LED_Status_k,send_k,running_k,_Lk1,_Lk2,_Lk3))
        threads.append(t) #creates the new thread number
        t.start()
    for t in threads:
        t.join()
    print('Multithreading has finished.')
        


In [11]:
def run_pynq_client_C(start_c,LED_Status_c,send_c,running_c,_Lc0,_Lc1,_Lc2,_Lc3):
    #global LED_Status, Send_Status, start_LED
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print('Press Button0 to Connect to Server.')
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            print('About to connect to Server.')
            sock.connect(('192.168.2.1', 12345))
            print('Connected to server!')
            with _Lc0:
                start_c.value = 1 #Allow Loop() to start with threading
            break
    while True:
        time.sleep(1)
        if base.buttons[0].read() != 0:
            sock.sendall(bytes('disconnect', 'utf-8'))
            with _Lc3:
                print('Client is sending shutdown message!')
                running_c.value = 0
            break
        with _Lc2:
            if send_c.value == 1:
                with _Lc1:
                    send = ''.join(map(str, LED_Status_c)) #Convert list into string variable (e.g. [0 0 2 3] -> '0023')
                sock.sendall(bytes(send, 'utf-8'))
                send_c.value = 0
    sock.close()

In [12]:
start_i = Value('i', 0) #SHARED! start_LED
send_i = Value('i',1) #SHARED! Send_Status
LED_Status_i = Array('i', [0, 0, 2, 3]) #SHARED
running_i = Value('i',1) #
_L0 = Lock() #Lock0 for Start
_L1 = Lock() #Lock1 For LED status
_L2 = Lock() #Lock2 for send
_L3 = Lock() #Lock for running (keeping LED() Flashing, stop when value changes from)


p_client = multiprocessing.Process(target=run_pynq_client_C,args=(start_i,LED_Status_i,send_i,running_i,_L0,_L1,_L2,_L3))
p_client.start()
os.system("taskset -p -c {} {}".format(0, p_client.pid))

p_GPIO_A_B = multiprocessing.Process(target=Start_Keypad,args=(start_i,LED_Status_i,send_i,running_i,_L0,_L1,_L2,_L3))
p_GPIO_A_B.start()
os.system("taskset -p -c {} {}".format(1, p_client.pid))

p_client.join();
print('Process Client with name, {}, is finished'.format(p_client.name))
p_GPIO_A_B.join();
print('Process GPIO with name, {}, is finished'.format(p_GPIO_A_B.name))

Press Button0 to Connect to Server.
About to connect to Server.
Connected to server!
New Input Detected:  147*
New status update to be sent
LED() temporarily terminated.
New Input Detected:  6
New status update to be sent
LED() temporarily terminated.
New Input Detected:  9#
New status update to be sent
LED() temporarily terminated.
New Input Detected:  B
New status update to be sent
LED() temporarily terminated.
New Input Detected:  2580
New status update to be sent
LED() temporarily terminated.
New Input Detected:  6
New status update to be sent
New Input Detected:  6
New status update to be sent
LED() temporarily terminated.
Client is sending shutdown message!
Process Client with name, Process-3, is finished
LED() temporarily terminated.
Multithreading has finished.
Process GPIO with name, Process-4, is finished
